In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import optim
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
x_data = pd.read_csv("../python-docker/swedbank/x_Swedbank_A_70_p.csv", sep=";", usecols = [i for i in range(70)])

In [3]:
y_data = pd.read_csv("../python-docker/swedbank/y_Swedbank_A_70.csv", sep=";")

In [4]:
x_data.head() 

,Swedbank_A-price-0,Swedbank_A-price-1,Swedbank_A-price-2,Swedbank_A-price-3,Swedbank_A-price-4,Swedbank_A-price-5,Swedbank_A-price-6,Swedbank_A-price-7,Swedbank_A-price-8,Swedbank_A-price-9,...,Swedbank_A-price-60,Swedbank_A-price-61,Swedbank_A-price-62,Swedbank_A-price-63,Swedbank_A-price-64,Swedbank_A-price-65,Swedbank_A-price-66,Swedbank_A-price-67,Swedbank_A-price-68,Swedbank_A-price-69
0,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,...,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14
1,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,...,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.20
2,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,...,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.20
3,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,...,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.20
4,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,...,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.14,145.20


In [5]:
y_data.head()

,15s,15sa,15ud,30s,30sa,30ud,45s,45sa,45ud,60s,...,180s,180sa,180ud,300s,300sa,300ud,600s,600sa,600ud,ts
0,145.14,145.0,0,145.14,145.070000,0,145.24,145.24,1,145.32,...,145.16,145.168,1,145.34,145.263,1,145.26,145.238333,1,1604390404
1,145.14,145.0,-1,145.00,145.046667,-1,145.24,145.24,1,145.32,...,145.16,145.168,-1,145.34,145.263,1,145.26,145.238333,1,1604390405
2,145.14,145.0,-1,145.00,145.046667,-1,145.24,145.24,1,145.32,...,145.16,145.168,-1,145.34,145.263,1,145.26,145.238333,1,1604390406
3,145.14,145.0,-1,145.00,145.046667,-1,145.24,145.24,1,145.32,...,145.16,145.168,-1,145.34,145.263,1,145.26,145.238333,1,1604390407
4,145.14,145.0,-1,145.00,145.046667,-1,145.24,145.24,1,145.32,...,145.16,145.168,-1,145.34,145.263,1,145.26,145.238333,1,1604390408


In [6]:
x_data.shape

(1809209, 70)

In [7]:
y_data = y_data["60ud"] #300s - 5min exakt price
y_data.shape

(1809209,)

In [34]:
w0 = sum(y_data == -1) / sum(y_data == 0)
w1 = 1
w2 = sum(y_data == 1) / sum(y_data == 0)

In [8]:
dtype = torch.FloatTensor
#dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [9]:
def splitData(xs, ys, trainRatio):
    t = round(len(xs)*trainRatio)
    
    train_data_x = torch.tensor(xs[:t].values, dtype=torch.float32)
    train_data_y = torch.tensor(ys[:t].values, dtype=torch.float32)
    
    d = round(len(xs[t:])/2)
    
    dev_data_x = torch.tensor(xs[t:][:d].values, dtype=torch.float32)
    dev_data_y = torch.tensor(ys[t:][:d].values, dtype=torch.float32)
    
    test_data_x = torch.tensor(xs[t:][d:].values, dtype=torch.float32)
    test_data_y = torch.tensor(ys[t:][d:].values, dtype=torch.float32)
    
    #print(test_data_y.shape)
    
    return TensorDataset(train_data_x, train_data_y), TensorDataset(dev_data_x, dev_data_y), list(zip(test_data_x, test_data_y))

In [10]:
train_data, dev_data, test_data = splitData(x_data, y_data, 0.8)

In [11]:
input_size=70
batch_size=128
nbr_epochs=5

In [12]:
#compute_unit = "cuda:0" if torch.cuda.is_available() else "cpu"
#device = torch.device('cuda:0')
device = torch.device("cpu")
#compute_unit = "cpu"

In [67]:
class StockModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 90).type(dtype)
        self.fc2 = nn.Linear(90, 70).type(dtype)
        self.fc3 = nn.Linear(70, 30).type(dtype)
        self.fc4 = nn.Linear(30, 10).type(dtype)
        self.fc5 = nn.Linear(10, 3).type(dtype)
        
        self.softmax = nn.LogSoftmax(dim=0)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return self.softmax(self.fc5(x))
    
    def eval(self, x):
        with torch.no_grad():
            y_ = self.forward(x)
            print(y_)
            return y_.argmax().item()
        
model = StockModel(input_size)
weight = torch.tensor([w0,w1,w2])
loss_fn = nn.NLLLoss(weight = weight)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [87]:
def evaluate_model(dev_data_loader, model, loss_fn, evalMode = False):
    losses = []
    n_correct = 0
    predictions = []
    with torch.no_grad():
        for x, y in dev_data_loader:
            
            #y = y.unsqueeze(0)
            y = y + 1
            y = y.long()
            pred = model(x)
            if not evalMode:
                loss = loss_fn(pred, y)
                losses.append(loss.item())
                n_correct += torch.sum(pred.argmax(dim=1) == y).item()
            else:
                n_correct += torch.sum(pred.argmax(dim=0) == y).item()
            predictions.append(pred)
        dev_avg_loss = sum(losses)/len(losses)    
        dev_acc = n_correct/len(dev_data)
    return dev_avg_loss, dev_acc, predictions

In [88]:
def train_model(model, train_data_loader, dev_data_loader, loss_fn, optimizer, epochrange, batchsize):
    for epoch in range(epochrange):
        losses = []
        n_correct = 0
        for x, y in train_data_loader:
            y = y + 1
            y = y.long()
            pred = model(x)
            loss = loss_fn(pred, y)
            print(pred)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()    
            n_correct += torch.sum(pred.argmax(dim=1) == y).item()
            #n_correct += torch.sum((pred > 0.5) == y.bool()).item()
            
        # Compute accuracy and loss in the entire training set
        train_avg_loss = sum(losses)/len(losses)    
        
        dev_avg_loss, dec_acc, _ = evaluate_model(dev_data_loader, model, loss_fn)
        
        train_acc = n_correct/len(train_data)
        
        # Display metrics
        display_str = 'Epoch {} '
        display_str += '\tLoss (train): {:.3f} '
        display_str += '\tAcc (train): {:.3f}% '
        display_str += '\tLoss (dev): {:.3f}'
        display_str += '\tAcc (dev): {:.3f}%'
        print(display_str.format(epoch, train_avg_loss, train_acc*100, dev_avg_loss, dec_acc*100))

In [89]:
train_data_loader = DataLoader(train_data, batch_size=batch_size)
dev_data_loader = DataLoader(dev_data, batch_size=batch_size)

In [90]:
model = model.to(device)
train_model(model, train_data_loader, dev_data_loader, loss_fn, optimizer, nbr_epochs, batch_size)

Epoch 0 	Loss (train): 4.803 	Acc (train): 57.795% 	Loss (dev): 4.801	Acc (dev): 58.030%
Epoch 1 	Loss (train): 4.793 	Acc (train): 57.840% 	Loss (dev): 4.800	Acc (dev): 57.919%


KeyboardInterrupt: 

In [ ]:
_,_, preds = evaluate_model(test_data, model, loss_fn, evalMode = True)

In [70]:
y_data_borpi = list(zip(*test_data))
print(preds[0])
items_plot = [y_data_borpi[1][t] for t in range(len(y_data_borpi[1]))]
plt.plot(list(range(len(preds))), preds)
plt.plot(list(range(len(items_plot))), items_plot)
axes = plt.gca()
axes.set_ylim([145,170])
#axes.set_xlim([125000,129000])
axes.set_xlim([176950,177500])
#axes.set_xlim([23800,24100])
plt.show()

NameError: name 'preds' is not defined